# Basic Statistical Matching Example

This example demonstrates the basic usage of py-statmatch for nearest neighbor matching.

In [ ]:
import pandas as pd
import numpy as np
from statmatch import nnd_hotdeck
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)
sns.set_style("whitegrid")

## Create Example Datasets

We'll create two datasets:
- **Donors**: Contains age, income, education, and job satisfaction (to be donated)
- **Recipients**: Contains age, income, and education (missing job satisfaction)

In [ ]:
# Create donor dataset
n_donors = 200
donors = pd.DataFrame(
    {
        "age": np.random.normal(40, 10, n_donors),
        "income": np.random.lognormal(
            10.5, 0.5, n_donors
        ),  # Log-normal for realistic income
        "education_years": np.random.normal(14, 3, n_donors),
        "job_satisfaction": np.random.beta(6, 4, n_donors)
        * 10,  # Beta for bounded variable
    }
)

# Round age and education to integers
donors["age"] = donors["age"].round().astype(int)
donors["education_years"] = (
    donors["education_years"].clip(8, 20).round().astype(int)
)

print("Donor dataset shape:", donors.shape)
donors.head()

In [ ]:
# Create recipient dataset
n_recipients = 100
recipients = pd.DataFrame(
    {
        "age": np.random.normal(38, 12, n_recipients),
        "income": np.random.lognormal(10.4, 0.6, n_recipients),
        "education_years": np.random.normal(13, 3, n_recipients),
    }
)

# Round age and education to integers
recipients["age"] = recipients["age"].round().astype(int)
recipients["education_years"] = (
    recipients["education_years"].clip(8, 20).round().astype(int)
)

print("Recipient dataset shape:", recipients.shape)
recipients.head()

## Visualize the Datasets

Let's visualize the distributions to understand the matching challenge:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Age distribution
axes[0].hist(donors["age"], alpha=0.5, label="Donors", bins=20)
axes[0].hist(recipients["age"], alpha=0.5, label="Recipients", bins=20)
axes[0].set_xlabel("Age")
axes[0].set_ylabel("Count")
axes[0].legend()
axes[0].set_title("Age Distribution")

# Income distribution
axes[1].hist(donors["income"], alpha=0.5, label="Donors", bins=20)
axes[1].hist(recipients["income"], alpha=0.5, label="Recipients", bins=20)
axes[1].set_xlabel("Income")
axes[1].legend()
axes[1].set_title("Income Distribution")

# Education distribution
axes[2].hist(
    donors["education_years"], alpha=0.5, label="Donors", bins=range(8, 22)
)
axes[2].hist(
    recipients["education_years"],
    alpha=0.5,
    label="Recipients",
    bins=range(8, 22),
)
axes[2].set_xlabel("Education Years")
axes[2].legend()
axes[2].set_title("Education Distribution")

plt.tight_layout()
plt.show()

## Perform Statistical Matching

Now let's use `nnd_hotdeck` to match recipients to donors:

In [ ]:
# Perform matching
result = nnd_hotdeck(
    data_rec=recipients,
    data_don=donors,
    match_vars=["age", "income", "education_years"],
    dist_fun="euclidean",
)

print("Matching complete!")
print(f"Average match distance: {result['dist.rd'].mean():.2f}")
print(f"Min distance: {result['dist.rd'].min():.2f}")
print(f"Max distance: {result['dist.rd'].max():.2f}")

## Create Fused Dataset

Now we'll create the fused dataset by adding the donated variable:

In [ ]:
# Create fused dataset
fused = recipients.copy()
fused["job_satisfaction"] = donors.iloc[result["noad.index"]][
    "job_satisfaction"
].values
fused["donor_id"] = result["noad.index"]
fused["match_distance"] = result["dist.rd"]

print("Fused dataset shape:", fused.shape)
fused.head()

## Analyze Match Quality

In [ ]:
# Distribution of match distances
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(result["dist.rd"], bins=30, edgecolor="black")
plt.xlabel("Match Distance")
plt.ylabel("Count")
plt.title("Distribution of Match Distances")

plt.subplot(1, 2, 2)
plt.scatter(fused["match_distance"], fused["job_satisfaction"], alpha=0.5)
plt.xlabel("Match Distance")
plt.ylabel("Donated Job Satisfaction")
plt.title("Match Quality vs Donated Value")

plt.tight_layout()
plt.show()

## Examine Specific Matches

Let's look at some specific matches to understand the process:

In [ ]:
# Look at best and worst matches
best_match_idx = result["dist.rd"].argmin()
worst_match_idx = result["dist.rd"].argmax()

print("BEST MATCH:")
print(f"Distance: {result['dist.rd'][best_match_idx]:.4f}")
print("\nRecipient:")
print(recipients.iloc[best_match_idx][["age", "income", "education_years"]])
print("\nMatched Donor:")
print(
    donors.iloc[result["noad.index"][best_match_idx]][
        ["age", "income", "education_years", "job_satisfaction"]
    ]
)

print("\n" + "=" * 50 + "\n")

print("WORST MATCH:")
print(f"Distance: {result['dist.rd'][worst_match_idx]:.4f}")
print("\nRecipient:")
print(recipients.iloc[worst_match_idx][["age", "income", "education_years"]])
print("\nMatched Donor:")
print(
    donors.iloc[result["noad.index"][worst_match_idx]][
        ["age", "income", "education_years", "job_satisfaction"]
    ]
)

## Compare Distance Metrics

Let's see how different distance metrics affect the matching:

In [ ]:
# Try different distance functions
distance_functions = ["euclidean", "manhattan", "cosine"]
results_by_distance = {}

for dist_fun in distance_functions:
    result = nnd_hotdeck(
        data_rec=recipients,
        data_don=donors,
        match_vars=["age", "income", "education_years"],
        dist_fun=dist_fun,
    )
    results_by_distance[dist_fun] = result["dist.rd"]

# Compare distance distributions
plt.figure(figsize=(12, 4))
for i, (dist_fun, distances) in enumerate(results_by_distance.items()):
    plt.subplot(1, 3, i + 1)
    plt.hist(distances, bins=30, edgecolor="black")
    plt.xlabel("Match Distance")
    plt.ylabel("Count")
    plt.title(f"{dist_fun.capitalize()} Distance")
    plt.text(
        0.05,
        0.95,
        f"Mean: {distances.mean():.2f}\nStd: {distances.std():.2f}",
        transform=plt.gca().transAxes,
        verticalalignment="top",
        bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5),
    )

plt.tight_layout()
plt.show()

## Summary

This example demonstrated:
1. Basic usage of `nnd_hotdeck` for statistical matching
2. Creating a fused dataset with donated variables
3. Analyzing match quality through distance distributions
4. Comparing different distance metrics

Key takeaways:
- Lower match distances indicate better matches
- The choice of distance metric affects matching results
- It's important to examine the distribution of match distances to assess overall matching quality
- Extreme cases (best/worst matches) can help identify potential issues